In [1]:
%matplotlib inline
from textblob import TextBlob, Word
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [2]:
import csv
import pandas as pd
with open('reviews_suomi.txt', encoding='latin1') as f:
  result = f.readlines()
  result = [line.rstrip('\n') for line in open('reviews_suomi.txt', encoding='latin1')]
  result
  print(result)

['Ei tarkastele ku 30min ja sit pitÃ¤Ã¤ sallia uudestaan et saa taustalla toimia eli iha p**** sovellus. Muutenkii koko korona hÃ¶ssÃ¶tys on iha turhaa. En haluis antaa yhtÃ¤kÃ¤Ã¤ tÃ¤hteÃ¤.', '"LÃ¤hipiirissÃ¤ Koronatartunta. Ei nÃ¤y sovelluksessa ollenkaan. Tuntikausia jo mennyt puhelimitse ilmoitetusta tartunnasta, kaveri ollut n.5-15m pÃ¤Ã¤ssÃ¤ meikÃ¤lÃ¤isestÃ¤ yli tunnin???"', 'Kun luin arvosteluja/kokemuksia niin en kyllÃ¤ voi luottaa koronavilkkuun yhtÃ¤Ã¤n. Ei taida olla mitÃ¤Ã¤n hyÃ¶tyÃ¤!', 'Ei toimi pÃ¤ivityksen jÃ¤lkeen! HyppÃ¤Ã¤ pois koko sovelluksesta kun sen yrittÃ¤Ã¤ avata.', '"Ei pÃ¤ivity automaattisesti. Joutuu kÃ¤ynnistÃ¤mÃ¤Ã¤n puhelimen uudestaan, kun sitten joskus muistaa tarkastaa tilanteen. TÃ¤nÃ¤Ã¤n 29.10. ja juuri Ã¤sken ""tarkastettu viimeksi 24.10."". Puhelin Huawei."', '"Ei toimi ilman gps-paikannusta vaikka niin kovin vÃ¤itetÃ¤Ã¤n. Adroid9, eikÃ¤ nÃ¤kÃ¶jÃ¤Ã¤n Andr10:skÃ¤Ã¤n."', 'Sovellus ei pÃ¤ivity automaattisesti eikÃ¤ asetuksista lÃ¶ydy pÃ¤ivitystÃ¤. PitÃ¤Ã

In [ ]:
# Text cleaning 
import nltk
nltk.download('punkt')
from string import punctuation
import re

def clean_sentence(sentence):
    sentence = re.sub(r"(?:\@|https?\://)\S+|\n+", "", sentence.lower())
    # Fix spelling errors in comments!
    sent = TextBlob(sentence)
    #sent = sent_suomi.translate(from_lang='fi', to='en')
    sent.correct()
    clean = ""
    for sentence in sent.sentences:    
        words = sentence.words
        # Remove punctuations
        words = [''.join(c for c in s if c not in punctuation) for s in words]
        words = [s for s in words if s]
        clean += " ".join(words)
        clean += ". "
    return clean
import csv
import pandas as pd
with open('reviews_suomi.txt', encoding='latin1') as f:
  result = f.readlines()
  result = [line.rstrip('\n') for line in open('reviews_suomi.txt', encoding='latin1')]
  result
result = [clean_sentence(x) for x in result]
result
type(result)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Men\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Check sentiment polarity of each sentence.
sentiment_scores = list()
i = 0
for sentence in result:
    line_suomi = TextBlob(sentence)
    line = line_suomi.translate(to='en')
    sentiment_scores.append(line.sentiment.polarity)
    if(i <= 10):
        print(sentence + ": POLARITY=" + str(line.sentiment.polarity))
        i += 1

In [ ]:
sns.distplot(sentiment_scores)

In [ ]:
fig,axes=plt.subplots(1,3)

In [ ]:
# Convert array of comments into a single string
comments = TextBlob(' '.join(result))
# Check out noun phrases, will be useful for frequent feature extraction
comments.noun_phrases

In [ ]:
#compactness pruning:
cleaned = list()
for phrase in comments.noun_phrases:
    count = 0
    for word in phrase.split():
        # Count the number of small words and words without an English definition
        if len(word) <= 2 or (not Word(word).definitions):
            count += 1
    # Only if the 'nonsensical' or short words DO NOT make up more than 40% (arbitrary) of the phrase add
    # it to the cleaned list, effectively pruning the ones not added.
    if count < len(phrase.split())*0.4:
        cleaned.append(phrase)
        
print("After compactness pruning:\nFeature Size:")
len(cleaned)

In [ ]:
for phrase in cleaned:    
    match = list()
    temp = list()
    word_match = list()
    for word in phrase.split():
        # Find common words among all phrases
        word_match = [p for p in cleaned if re.search(word, p) and p not in word_match]
        # If the size of matched phrases set is smaller than 30% of the cleaned phrases, 
        # then consider the phrase as non-redundant.
        if len(word_match) <= len(cleaned)*0.3 :
            temp.append(word)
            match += word_match
            
    phrase = ' '.join(temp)
#     print("Match for " + phrase + ": " + str(match))

    if len(match) >= len(cleaned)*0.1 :
        # Redundant feature set, since it contains more than 10% of the number of phrases. 
        # Prune all matched features.
        for feature in match:
            if feature in cleaned:
                cleaned.remove(feature)
            
        # Add largest length phrase as feature
        cleaned.append(max(match, key=len))
        

print("After redundancy pruning:\nFeature Size:" + str(len(cleaned)))
print("Cleaned features:")
cleaned


In [ ]:
from nltk.corpus import stopwords
feature_count = dict()
for phrase in cleaned:
    count = 0
    for word in phrase.split():
        if word not in stopwords.words('finnish'):
            count += comments.words.count(word)
    
    print(phrase + ": " + str(count))
    feature_count[phrase] = count

In [ ]:
# Select frequent feature threshold as (max_count)/100 
# This is an arbitrary decision as of now.
counts = list(feature_count.values())
features = list(feature_count.keys())
threshold = len(comments.noun_phrases)/100
#threshold=66

print("Threshold:" + str(threshold))

frequent_features = list()

for feature, count in feature_count.items():
    if count >= threshold:
        frequent_features.append(feature)
print(' Features:')
frequent_features=frequent_features[0:24]
frequent_features

In [ ]:
def nltk_sentiment(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score
#b=dataset.values.T.tolist()
#print(b)
nltk_results = [nltk_sentiment(row) for row in frequent_features]
#print(nltk_results)
results_df = pd.DataFrame(nltk_results)
#print(results_df)
text_df = pd.DataFrame(frequent_features)
#print(text_df)
nltk_df = text_df.join(results_df)
#nltk_df1=nltk_df[[0,'neu']]
#print(nltk_df)


In [ ]:
nltk_df.head(24)

In [ ]:
newdf=nltk_df[0]
newdf=pd.DataFrame({'features':nltk_df[0],'pos':nltk_df['pos'],'neg':nltk_df['neg']})
newdf.pos=newdf.pos#+0.2
newdf.neg=newdf.neg#-0.2
newdf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
newdf

#noun=['positive attitude','good job knowledge','team player','customer acquisition','good visibility','back office process','successful launch','soft skills','suitable candidates','core issues']
#pos_l=[0.71,0.42,0.3,0.2,0.2,0.6,0.2,0.5,0.43,0]
#neg_l=[-0.1,0,-0.300,0,-0.1,0,-0.1,-0.2,0,-0.3]

pos=newdf[0:10]['pos']
neg=newdf[0:10]['neg']

# data to plot
n_groups = 5
positive =newdf['pos'].head(5) 
negative =newdf['neg'].head(5)
 
# create plot
fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.3
opacity = 1
 
rects1 = plt.bar(index, positive, bar_width,
alpha=opacity,
color='b',
label='positive sentiments')
 
rects2 = plt.bar(index + bar_width, negative, bar_width,
alpha=opacity,
color='r',
label='negative sentiments')
 
plt.xlabel('Features')
plt.ylabel('sentiment value')
plt.title('Top features and its sentiment')
plt.xticks(index + bar_width, newdf['features'].head(5))
plt.legend()
fig.set_size_inches(20, 15)
plt.show()

In [ ]:
absa_list = dict()
print(absa_list)
# For each frequent feature
for f in frequent_features:
    # For each comment
    absa_list[f] = list()
    for comment in result:
        blob = TextBlob(comment)
        # For each sentence of the comment
        for sentence in blob.sentences:
            # Search for frequent feature 'f'
            q = '|'.join(f.split())
            if re.search(r'\w*(' + str(q) + ')\w*', str(sentence)):
                absa_list[f].append(sentence)


print("Aspect Specific sentences:")
absa_list

In [ ]:
scores = list()
absa_scores = dict()
for k, v in absa_list.items():
    absa_scores[k] = list()
    for sent in v:
        score = sent.sentiment.polarity
        scores.append(score)
        absa_scores[k].append(score)

In [ ]:
# Now that we have all the scores, let's plot them!
# For comparison, we replot the previous global sentiment polarity plot
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(20, 10))
plot1 = sns.distplot(scores, ax=ax1)

ax1.set_title('Aspect wise scores')
ax1.set_xlabel('Sentiment Polarity')
ax1.set_ylabel('# of comments')

ax2.set_title('Comment wise scores')
ax2.set_xlabel('Sentiment Polarity')
ax2.set_ylabel('# of comments')

plot2 = sns.distplot(sentiment_scores, ax=ax2)

In [ ]:
# Create data values for stripplot and boxplot
vals = dict()
vals["aspects"] = list()
vals["scores"] = list()
for k, v in absa_scores.items():
    for score in v:
        vals["aspects"].append(k)
        vals["scores"].append(score)

In [ ]:
fig, ax1 = plt.subplots(figsize=(30, 10))

color = sns.color_palette("Blues", 6)
plt.xticks(rotation=90)
sns.set_context("paper", font_scale=3) 
sns.boxplot(x="aspects", y="scores", data=vals, palette=color, ax=ax1)

In [ ]:
color = sns.color_palette("Accent", 10)
fig, ax1 = plt.subplots(figsize=(30, 10))
plt.xticks(rotation=90)
sns.set_context("paper", font_scale=5) 
sns.stripplot(y="aspects", x="scores",data=vals, palette=color)